In [91]:
import pandas as pd
import numpy as np
from faker import Faker
import random
import openpyxl
from sklearn.pipeline import Pipeline
from datetime import datetime, timedelta
from sklearn.base import BaseEstimator, TransformerMixin

# **Generate Fake Data**

In [92]:
def random_date(start, end):
    return start + timedelta(days=random.randint(0, int((end - start).days)))


np.random.seed(42)
num_records = 10000

loan_ids = np.arange(1, num_records + 1)
disbursement_dates = [
    random_date(datetime(2020, 1, 1), datetime(2023, 1, 1)) for _ in range(num_records)
]
expire_dates = [d + timedelta(days=random.randint(30, 365)) for d in disbursement_dates]
is_employed = np.random.choice([True, False, np.nan], num_records, p=[0.7, 0.25, 0.05])
loan_amounts = np.random.choice(
    [np.nan, *np.random.uniform(-1000, 50000, num_records)], num_records
)
number_of_defaults = np.random.choice(
    [np.nan, *np.random.randint(0, 10, num_records)], num_records
)
outstanding_balances = np.random.choice(
    [np.nan, *np.random.uniform(0, 30000, num_records)], num_records
)
interest_rates = np.random.choice(
    [np.nan, *np.random.uniform(0.01, 0.25, num_records)], num_records
)
ages = np.random.choice([np.nan, *np.random.randint(18, 65, num_records)], num_records)
remaining_terms = np.random.choice(
    [np.nan, *np.random.randint(1, 60, num_records)], num_records
)
salaries = np.random.choice(
    [np.nan, *np.random.uniform(-10000, 20000, num_records)], num_records
)
loan_statuses = np.random.choice(
    ["Default", "Non-Default", np.nan], num_records, p=[0.3, 0.65, 0.05]
)
sectors = np.random.choice(
    ["Agriculture", "Manufacturing", "Services", "IT", "Retail"], num_records
)
currencies = np.random.choice(["USD", "EUR", "ZWL", "GBP", "AUD"], num_records)
employee_sectors = np.random.choice(
    ["Public", "Private", "Self-employed", "Unemployed"], num_records
)
statuses = np.random.choice(["Active", "Inactive"], num_records)

loan_data = pd.DataFrame(
    {
        "loan_id": loan_ids,
        "disbursement_date": disbursement_dates,
        "expire_date": expire_dates,
        "is_employed": is_employed,
        "loan_amount": loan_amounts,
        "number_of_defaults": number_of_defaults,
        "outstanding_balance": outstanding_balances,
        "interest_rate": interest_rates,
        "age": ages,
        "remaining_term": remaining_terms,
        "salary": salaries,
        "sector": sectors,
        "currency": currencies,
        # "employee_sector": employee_sectors,
        # "status": statuses,
        "loan_status": loan_statuses,
    }
)

##### Create Duplicates
num_duplicates = 100
duplicates = loan_data.sample(n=num_duplicates, replace=True).reset_index(drop=True)
loan_data = pd.concat([loan_data, duplicates], ignore_index=True)

In [93]:
loan_data.head(3)

,loan_id,disbursement_date,expire_date,is_employed,loan_amount,number_of_defaults,outstanding_balance,interest_rate,age,remaining_term,salary,sector,currency,loan_status
0,1,2021-02-12,2021-08-30,1.0,28982.170288,4.0,29869.899084,0.112340,38.0,49.0,4908.585021,Agriculture,USD,Non-Default
1,2,2021-02-25,2021-11-09,NaN,27464.842065,5.0,27408.000182,0.128941,30.0,30.0,2283.627180,Agriculture,EUR,Non-Default
2,3,2020-03-17,2021-01-07,0.0,24469.175558,2.0,19078.850400,0.224119,63.0,17.0,4093.206305,Services,USD,Non-Default


In [94]:
loan_data.shape

(10100, 14)

# Data Cleanign steps

- Data cleaning is the process of ensuring that data is in the proper format, *making it suitable for analysis and modeling*

### 1.Check missing values 

In [95]:
category_columns = loan_data.select_dtypes("object").columns
category_columns

Index(['sector', 'currency', 'loan_status'], dtype='object')

In [96]:
data = loan_data.copy()

In [97]:
data.columns = data.columns.str.strip()

In [98]:
duplicates = data.loc[data.duplicated(keep=False)].sort_values("loan_id")
duplicates

,loan_id,disbursement_date,expire_date,is_employed,loan_amount,number_of_defaults,outstanding_balance,interest_rate,age,remaining_term,salary,sector,currency,loan_status
312,313,2022-06-26,2022-12-03,1.0,20887.773416,9.0,4965.231331,0.126386,37.0,32.0,5918.435398,Manufacturing,ZWL,Non-Default
10018,313,2022-06-26,2022-12-03,1.0,20887.773416,9.0,4965.231331,0.126386,37.0,32.0,5918.435398,Manufacturing,ZWL,Non-Default
579,580,2021-09-20,2022-06-20,1.0,35175.516492,8.0,28213.374035,0.241272,52.0,50.0,-6876.003554,Services,ZWL,Non-Default
10081,580,2021-09-20,2022-06-20,1.0,35175.516492,8.0,28213.374035,0.241272,52.0,50.0,-6876.003554,Services,ZWL,Non-Default
10034,595,2020-09-04,2021-07-03,0.0,31323.209397,3.0,1511.835714,0.014322,52.0,22.0,7798.739957,Agriculture,ZWL,Non-Default
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10007,9798,2022-09-24,2022-11-18,1.0,10684.947309,6.0,26704.261346,0.121897,27.0,1.0,17557.191193,Manufacturing,EUR,nan
9946,9947,2020-12-12,2021-06-14,1.0,12806.860668,2.0,22788.492854,0.213088,33.0,12.0,-9136.527167,Retail,AUD,Default
10088,9947,2020-12-12,2021-06-14,1.0,12806.860668,2.0,22788.492854,0.213088,33.0,12.0,-9136.527167,Retail,AUD,Default
10003,9949,2020-05-01,2020-07-23,1.0,35573.093762,8.0,11054.830129,0.139636,28.0,33.0,13652.848693,Manufacturing,USD,Non-Default


In [99]:
def duplicates_check(df: pd.DataFrame):
    return data.loc[data.duplicated(keep=False)].sort_values("loan_id")

In [100]:
num_columns = [
    "loan_amount",
    "number_of_defaults",
    "outstanding_balance",
    "interest_rate",
    "age",
    "remaining_term",
    "salary",
]

In [101]:
data.head()

,loan_id,disbursement_date,expire_date,is_employed,loan_amount,number_of_defaults,outstanding_balance,interest_rate,age,remaining_term,salary,sector,currency,loan_status
0,1,2021-02-12,2021-08-30,1.0,28982.170288,4.0,29869.899084,0.112340,38.0,49.0,4908.585021,Agriculture,USD,Non-Default
1,2,2021-02-25,2021-11-09,NaN,27464.842065,5.0,27408.000182,0.128941,30.0,30.0,2283.627180,Agriculture,EUR,Non-Default
2,3,2020-03-17,2021-01-07,0.0,24469.175558,2.0,19078.850400,0.224119,63.0,17.0,4093.206305,Services,USD,Non-Default
3,4,2021-02-16,2021-06-26,1.0,36218.404504,9.0,5269.500606,0.125747,30.0,7.0,13755.258629,Retail,GBP,Default
4,5,2022-09-11,2023-02-14,1.0,45353.259460,6.0,22912.296526,0.142642,40.0,10.0,14089.467579,Retail,EUR,Default


In [102]:
def check_missing_values(df: pd.DataFrame):
    return data.loc[data.isnull().any(axis=1)]

In [103]:
data[data.isnull().any(axis=1)]

,loan_id,disbursement_date,expire_date,is_employed,loan_amount,number_of_defaults,outstanding_balance,interest_rate,age,remaining_term,salary,sector,currency,loan_status
1,2,2021-02-25,2021-11-09,NaN,27464.842065,5.0,27408.000182,0.128941,30.0,30.0,2283.627180,Agriculture,EUR,Non-Default
11,12,2022-03-22,2022-10-28,NaN,9899.361717,1.0,4842.795849,0.192387,27.0,38.0,-2135.138884,IT,EUR,Non-Default
34,35,2022-04-11,2022-11-29,NaN,10926.266823,3.0,14616.721703,0.189646,24.0,24.0,2609.446486,Manufacturing,EUR,Non-Default
50,51,2021-10-24,2022-10-12,NaN,34120.093785,0.0,9980.403155,0.243796,39.0,2.0,18455.388926,Agriculture,AUD,Default
69,70,2022-10-16,2023-03-15,NaN,23488.845535,7.0,15711.060298,0.209259,27.0,14.0,-6018.926070,IT,USD,Non-Default
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10005,6267,2022-12-15,2023-08-19,NaN,43439.224403,1.0,13526.688453,0.042392,34.0,41.0,-3852.851532,Manufacturing,ZWL,Default
10029,4678,2021-05-11,2021-12-26,NaN,33955.737775,9.0,10839.297775,0.073503,56.0,59.0,-8005.482590,IT,ZWL,Non-Default
10058,1617,2022-11-07,2023-02-14,NaN,3942.330169,7.0,22910.580293,0.134743,31.0,12.0,-3302.427098,IT,ZWL,Default
10071,2496,2021-01-01,2021-08-17,1.0,17358.037769,1.0,6242.926831,0.194841,51.0,NaN,8746.591592,Agriculture,GBP,Default


In [104]:
class CheckMissingValues(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.errors = pd.DataFrame()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        missing_counts = X.isnull().sum()
        missing_counts = missing_counts[missing_counts > 0]
        self.errors = (
            missing_counts.to_frame(name="Missing Values")
            .rename_axis("Column")
            .reset_index()
            .sort_values("Missing Values", ascending=False)
        )
        return self.errors

In [105]:
missing = CheckMissingValues()
frms = missing.fit_transform(data)
frms

,Column,Missing Values
0,is_employed,478
6,remaining_term,3
3,outstanding_balance,2
2,number_of_defaults,1
1,loan_amount,1
4,interest_rate,1
5,age,1


In [106]:
check_missing_values = data.loc[data.isnull().any(axis=1)]
check_missing_values

,loan_id,disbursement_date,expire_date,is_employed,loan_amount,number_of_defaults,outstanding_balance,interest_rate,age,remaining_term,salary,sector,currency,loan_status
1,2,2021-02-25,2021-11-09,NaN,27464.842065,5.0,27408.000182,0.128941,30.0,30.0,2283.627180,Agriculture,EUR,Non-Default
11,12,2022-03-22,2022-10-28,NaN,9899.361717,1.0,4842.795849,0.192387,27.0,38.0,-2135.138884,IT,EUR,Non-Default
34,35,2022-04-11,2022-11-29,NaN,10926.266823,3.0,14616.721703,0.189646,24.0,24.0,2609.446486,Manufacturing,EUR,Non-Default
50,51,2021-10-24,2022-10-12,NaN,34120.093785,0.0,9980.403155,0.243796,39.0,2.0,18455.388926,Agriculture,AUD,Default
69,70,2022-10-16,2023-03-15,NaN,23488.845535,7.0,15711.060298,0.209259,27.0,14.0,-6018.926070,IT,USD,Non-Default
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10005,6267,2022-12-15,2023-08-19,NaN,43439.224403,1.0,13526.688453,0.042392,34.0,41.0,-3852.851532,Manufacturing,ZWL,Default
10029,4678,2021-05-11,2021-12-26,NaN,33955.737775,9.0,10839.297775,0.073503,56.0,59.0,-8005.482590,IT,ZWL,Non-Default
10058,1617,2022-11-07,2023-02-14,NaN,3942.330169,7.0,22910.580293,0.134743,31.0,12.0,-3302.427098,IT,ZWL,Default
10071,2496,2021-01-01,2021-08-17,1.0,17358.037769,1.0,6242.926831,0.194841,51.0,NaN,8746.591592,Agriculture,GBP,Default


In [107]:
def check_invalid_dates(df: pd.DataFrame):
    dates_invalid = df[df["disbursement_date"] > df["expire_date"]]
    return dates_invalid

In [108]:
dates_invalid = data[data["disbursement_date"] > data["expire_date"]]
dates_invalid

,loan_id,disbursement_date,expire_date,is_employed,loan_amount,number_of_defaults,outstanding_balance,interest_rate,age,remaining_term,salary,sector,currency,loan_status


In [109]:
mandatory_columns = [
    "loan_id",
    "disbursement_date",
    "expire_date",
    "is_employed",
    "loan_amount",
    "number_of_defaults",
    "outstanding_balance",
    "interest_rate",
    "age",
    "remaining_term",
    "salary",
    "sector",
    "currency",
    "employee_sector",
    "status",
    "loan_status",
]

In [110]:
def check_mandatory_columns(df: pd.DataFrame, mandatory_columns: list) -> list:
    missing_columns = [col for col in mandatory_columns if col not in df.columns]
    if missing_columns:
        return missing_columns
    else:
        return None

In [111]:
class MandatoryColumns(BaseEstimator, TransformerMixin):
    def __init__(self, mandatory_columns):
        self.mandatory_columns = mandatory_columns
        self.errors = None

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        self.errors = [col for col in self.mandatory_columns if col not in X.columns]
        return X

    def get_errors(self):
        return self.errors

In [112]:
num_columns_ck = ["loan_amount", "interest_rate", "age", "salary"]

In [113]:
conditions = [(data[col] < 0) | (data[col] == 0) for col in num_columns_ck]
mask = pd.concat(conditions, axis=1).any(axis=1)
check_negative_amounts_and_zeros = data[mask]
check_negative_amounts_and_zeros

,loan_id,disbursement_date,expire_date,is_employed,loan_amount,number_of_defaults,outstanding_balance,interest_rate,age,remaining_term,salary,sector,currency,loan_status
5,6,2021-08-24,2022-02-16,1.0,5974.506204,0.0,2492.023710,0.014792,63.0,47.0,-1529.950804,IT,ZWL,Non-Default
6,7,2021-05-03,2021-06-12,1.0,21934.173342,0.0,9747.824979,0.143750,22.0,36.0,-8570.523443,IT,ZWL,Non-Default
9,10,2020-10-30,2021-05-27,0.0,41763.628792,0.0,5985.145076,0.110431,21.0,31.0,-2405.424979,Agriculture,USD,Non-Default
10,11,2020-06-27,2020-10-09,1.0,31032.257305,8.0,8698.407086,0.098271,22.0,29.0,-2592.699823,Retail,AUD,Non-Default
11,12,2022-03-22,2022-10-28,NaN,9899.361717,1.0,4842.795849,0.192387,27.0,38.0,-2135.138884,IT,EUR,Non-Default
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10089,4604,2021-02-07,2022-01-06,1.0,8089.888044,9.0,16659.070767,0.227382,54.0,34.0,-9477.399211,IT,ZWL,Non-Default
10093,4904,2022-03-29,2022-08-09,1.0,19832.856833,6.0,3211.339731,0.045099,55.0,30.0,-8205.592246,Services,ZWL,Default
10094,712,2021-12-04,2022-05-29,1.0,10216.151950,1.0,26682.023733,0.096406,43.0,26.0,-908.600442,Retail,ZWL,Default
10096,4788,2021-09-18,2021-12-25,0.0,32633.463593,9.0,16284.579850,0.069643,59.0,32.0,-8419.546504,IT,EUR,Default


In [114]:
def check_negative_amounts_and_zeros(df: pd.DataFrame):
    conditions = [(data[col] < 0) | (data[col] == 0) for col in num_columns_ck]
    mask = pd.concat(conditions, axis=1).any(axis=1)
    negative_amounts_and_zeros = data[mask]
    negative_amounts_and_zeros
    return negative_amounts_and_zeros

In [115]:
data[num_columns] = data[num_columns].apply(pd.to_numeric, errors="coerce")
not_converted_num = data.loc[data[num_columns].isnull().any(axis=1)]

In [116]:
def converted_num(df: pd.DataFrame):
    num_columns = [
        "loan_amount",
        "number_of_defaults",
        "outstanding_balance",
        "interest_rate",
        "age",
        "remaining_term",
        "salary",
    ]
    data[num_columns] = data[num_columns].apply(pd.to_numeric, errors="coerce")
    not_converted_num = data.loc[data[num_columns].isnull().any(axis=1)]
    return not_converted_num

In [117]:
not_converted_num

,loan_id,disbursement_date,expire_date,is_employed,loan_amount,number_of_defaults,outstanding_balance,interest_rate,age,remaining_term,salary,sector,currency,loan_status
427,428,2021-06-30,2022-06-01,0.0,49449.532816,NaN,18552.233004,0.043065,35.0,25.0,19121.373969,IT,USD,Non-Default
1159,1160,2022-06-20,2023-01-17,1.0,30245.723605,3.0,4089.793095,NaN,30.0,28.0,-6032.421389,Agriculture,AUD,Non-Default
2495,2496,2021-01-01,2021-08-17,1.0,17358.037769,1.0,6242.926831,0.194841,51.0,NaN,8746.591592,Agriculture,GBP,Default
2498,2499,2021-01-25,2021-11-18,NaN,48750.015415,6.0,NaN,0.028080,27.0,6.0,7509.425339,Retail,AUD,Default
2837,2838,2022-06-02,2023-05-27,1.0,23774.989832,4.0,25581.870683,0.167046,31.0,NaN,-9841.307132,Retail,AUD,Non-Default
3241,3242,2021-06-26,2022-02-11,1.0,10968.860177,4.0,22488.176453,0.213830,NaN,11.0,6929.077248,Retail,AUD,Non-Default
5483,5484,2020-07-02,2021-01-15,0.0,47403.951535,2.0,NaN,0.040812,44.0,7.0,11909.320534,Manufacturing,EUR,Non-Default
7381,7382,2021-07-02,2021-10-26,1.0,NaN,6.0,16856.131988,0.226779,32.0,55.0,19476.052291,Retail,EUR,Default
10071,2496,2021-01-01,2021-08-17,1.0,17358.037769,1.0,6242.926831,0.194841,51.0,NaN,8746.591592,Agriculture,GBP,Default


In [118]:
dates_columns = loan_data.filter(regex="date").columns
category_columns = loan_data.select_dtypes("object").columns

In [119]:
for column in dates_columns:
    data[column] = pd.to_datetime(data[column], format="%d/%m/%Y", errors="coerce")

In [120]:
not_converted_dates = data.loc[data[dates_columns].isnull().any(axis=1)]
not_converted_dates

,loan_id,disbursement_date,expire_date,is_employed,loan_amount,number_of_defaults,outstanding_balance,interest_rate,age,remaining_term,salary,sector,currency,loan_status


In [121]:
def converted_dates(df: pd.DataFrame) -> pd.DataFrame:
    dates_columns = df.filter(regex="date").columns
    df[dates_columns] = df[dates_columns].apply(
        lambda col: pd.to_datetime(col, format="%d/%m/%Y", errors="coerce")
    )
    not_converted_dates = df[df[dates_columns].isnull().any(axis=1)]
    return not_converted_dates

In [122]:
class DateConverter(BaseEstimator, TransformerMixin):
    def __init__(self, date_formats=None):
        self.errors = None
        self.date_formats = date_formats or [
            "%d/%m/%Y",
            "%Y-%m-%d",
            "%m/%d/%Y",
            "%d-%m-%Y",
            "%Y.%m.%d",
        ]

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        dates_columns = X.filter(regex="date").columns
        X_temp = X.copy()

        not_converted_dates = (
            X_temp[dates_columns]
            .apply(
                lambda col: pd.to_datetime(
                    col, format=self.date_formats[0], errors="coerce"
                )
            )
            .loc[lambda df: df.isnull().any(axis=1)]
        )

        self.errors = not_converted_dates
        return not_converted_dates

In [123]:
mandatory_columns

['loan_id',
 'disbursement_date',
 'expire_date',
 'is_employed',
 'loan_amount',
 'number_of_defaults',
 'outstanding_balance',
 'interest_rate',
 'age',
 'remaining_term',
 'salary',
 'sector',
 'currency',
 'employee_sector',
 'status',
 'loan_status']

In [124]:
class ConvertedNumeric(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X):
        return self

    def transform(self, X):
        X = data.copy()
        num_columns = [
            "loan_amount",
            "number_of_defaults",
            "outstanding_balance",
            "interest_rate",
            "age",
            "remaining_term",
            "salary",
        ]
        X[num_columns] = X[num_columns].apply(pd.to_numeric, errors="coerce")
        not_converted_num = X.loc[X[num_columns].isnull().any(axis=1)]
        return not_converted_num

In [125]:
class CheckNegativeAmountsAndZeros(BaseEstimator, TransformerMixin):
    def __init__(self, num_columns_ck=None):
        self.num_columns_ck = num_columns_ck or [
            "loan_amount",
            "interest_rate",
            "age",
            "salary",
        ]
        self.errors = None

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        mask = X[self.num_columns_ck].apply(lambda col: (col <= 0)).any(axis=1)
        negative_amounts_and_zeros = X[mask]
        return negative_amounts_and_zeros

In [126]:
class CheckDuplicates(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.errors = None

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        duplicates = X[X.duplicated(keep=False)].sort_values("loan_id")
        return duplicates

In [127]:
data[data["disbursement_date"] > data["expire_date"]]

,loan_id,disbursement_date,expire_date,is_employed,loan_amount,number_of_defaults,outstanding_balance,interest_rate,age,remaining_term,salary,sector,currency,loan_status


In [128]:
class CheckInvalidDates(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.errors = None

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        invalid_dates = X[X["disbursement_date"] > X["expire_date"]]
        self.errors = invalid_dates
        return invalid_dates

In [129]:
data.columns

Index(['loan_id', 'disbursement_date', 'expire_date', 'is_employed',
       'loan_amount', 'number_of_defaults', 'outstanding_balance',
       'interest_rate', 'age', 'remaining_term', 'salary', 'sector',
       'currency', 'loan_status'],
      dtype='object')

In [130]:
pipeline = Pipeline(
    [
        ("mandatory_columns", MandatoryColumns(mandatory_columns=mandatory_columns)),
        ("check_missing_values", CheckMissingValues()),
        ("date_converter", DateConverter()),
        # ('check_invalid_dates', CheckInvalidDates()),
        ("convert_numeric", ConvertedNumeric()),
        ("check_negative_amounts_and_zeros", CheckNegativeAmountsAndZeros()),
        ("check_duplicates", CheckDuplicates()),
    ]
)

In [131]:
pipeline.fit(data)

Pipeline(steps=[('mandatory_columns',
                 MandatoryColumns(mandatory_columns=['loan_id',
                                                     'disbursement_date',
                                                     'expire_date',
                                                     'is_employed',
                                                     'loan_amount',
                                                     'number_of_defaults',
                                                     'outstanding_balance',
                                                     'interest_rate', 'age',
                                                     'remaining_term', 'salary',
                                                     'sector', 'currency',
                                                     'employee_sector',
                                                     'status',
                                                     'loan_status'])),
                ('check_missing_values', CheckMissingValues()),
                ('date_converter',
                 DateConverter(date_formats=['%d/%m/%Y', '%Y-%m-%d', '%m/%d/%Y',
                                             '%d-%m-%Y', '%Y.%m.%d'])),
                ('convert_numeric', ConvertedNumeric()),
                ('check_negative_amounts_and_zeros',
                 CheckNegativeAmountsAndZeros(num_columns_ck=['loan_amount',
                                                              'interest_rate',
                                                              'age',
                                                              'salary'])),
                ('check_duplicates', CheckDuplicates())])

In [132]:
df_cleaned = data.copy()
pipeline.fit(df_cleaned)
pipeline.transform(df_cleaned)

,loan_id,disbursement_date,expire_date,is_employed,loan_amount,number_of_defaults,outstanding_balance,interest_rate,age,remaining_term,salary,sector,currency,loan_status


In [133]:
with pd.ExcelWriter("data_issues.xlsx") as writer:
    mandatory_errors = pipeline.named_steps["mandatory_columns"].get_errors()
    if mandatory_errors is not None:
        pd.DataFrame(mandatory_errors, columns=["Missing Mandatory Columns"]).to_excel(
            writer, sheet_name="Missing Mandatory Columns"
        )
    else:
        pd.DataFrame(columns=["Missing Mandatory Columns"]).to_excel(
            writer, sheet_name="Missing Mandatory Columns", index=False
        )

    missing_values = pipeline.named_steps["check_missing_values"].errors
    if missing_values is not None and not missing_values.empty:
        missing_values.to_excel(writer, sheet_name="Missing Values")
    else:
        pd.DataFrame(columns=["Column", "Missing Values"]).to_excel(
            writer, sheet_name="Missing Values", index=False
        )

    invalid_dates = pipeline.named_steps["date_converter"].errors
    if invalid_dates is not None and not invalid_dates.empty:
        invalid_dates.to_excel(writer, sheet_name="Invalid Date Conversion")
    else:
        pd.DataFrame(columns=["Invalid Dates"]).to_excel(
            writer, sheet_name="Invalid Date Conversion", index=False
        )

    numeric_conversion_issues = pipeline.named_steps["convert_numeric"].transform(
        df_cleaned
    )
    if numeric_conversion_issues is not None and not numeric_conversion_issues.empty:
        numeric_conversion_issues.to_excel(
            writer, sheet_name="Numeric Conversion Issues"
        )
    else:
        pd.DataFrame(columns=["Numeric Conversion Issues"]).to_excel(
            writer, sheet_name="Numeric Conversion Issues", index=False
        )

    negative_amounts_and_zeros = pipeline.named_steps[
        "check_negative_amounts_and_zeros"
    ].transform(df_cleaned)
    if negative_amounts_and_zeros is not None and not negative_amounts_and_zeros.empty:
        negative_amounts_and_zeros.to_excel(
            writer, sheet_name="Negative Amounts and Zeros"
        )
    else:
        pd.DataFrame(columns=["Negative Amounts and Zeros"]).to_excel(
            writer, sheet_name="Negative Amounts and Zeros", index=False
        )

    duplicates = pipeline.named_steps["check_duplicates"].transform(df_cleaned)
    if duplicates is not None and not duplicates.empty:
        duplicates.to_excel(writer, sheet_name="Duplicates")
    else:
        pd.DataFrame(columns=["Duplicates"]).to_excel(
            writer, sheet_name="Duplicates", index=False
        )